In [14]:
import numpy as np
import random

In [15]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [16]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(12, 1), (14, 1), (11, 3), (9, 2), (11, 2), (10, 0), (13, 2), (10, 2)]
[(9, 0), (11, 0), (12, 0), (12, 3), (14, 0), (13, 3), (12, 2), (10, 3)]


In [17]:
class Player():
    def __init__(self, name):
        
        self.name = name
        self.cards = None
    
    ### -------------------------------------------------------------
    
    ### TO BE IMPLEMENTED - player's strategy 
    ### input: declared card, i.e., the card which is supposed
    ### to be the top card of the pile: If None - you can put any card you want because
    ### (a) it is the first turn (pile is empty) or (b) some cards were drawn in the previous turn)
    ### output: - player's true decision, player's declaration (if not equal - (s)he cheats)
    
    def putCard(self, declared_card):
        ### DO NOT REMOVE TRUE CARD cards.remove!!!
        ### return an object (not id): self.cards[id], not id
        ### for instance: return self.cards[0], self.cards[0] 
        ### IMPORTANT: If you want to draw cards instead of put, return "draw"
        ### for instance: return "draw" 
        return self.cards[0], self.cards[0] 
    
    ### TO BE IMPLEMENTED - Decide whether to check or not opponent's move (return True or False)
    def checkCard(self, opponent_declaration):
        pass
    
    ### Notification sent at the end of a round
    ### One may implement this method, capture data, and use it to get extra info
    ### -- checked = TRUE -> someone checked. If FALSE, the remaining inputs do not play any role
    ### -- iChecked = TRUE -> I decided to check my opponent (so it was my turn); 
    ###               FALSE -> my opponent checked and it was his turn
    ### -- iDrewCards = TRUE -> I drew cards (so I checked but was wrong or my opponent checked and was right); 
    ###                 FALSE -> otherwise
    ### -- revealedCard - some card (X, Y). Only if I checked.
    ### -- noTakenCards - number of taken cards
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        if log: print("Feedback = " + self.name + " : checked this turn = " + str(checked) +
              "; I checked = " + str(iChecked) + "; I drew cards = " + 
                      str(iDrewCards) + "; revealed card = " + 
                      str(revealedCard) + "; number of taken cards = " + str(noTakenCards))
    
    
    ### -------------------------------------------------------------
    
    ### Init player's hand
    def startGame(self, cards):
        self.cards = cards
    
    ### Add some cards to player's hand (if (s)he checked opponent's move, but (s)he was wrong)
    def takeCards(self, cards_to_take):
        self.cards = self.cards + cards_to_take

In [18]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

In [19]:
class Game():
    def __init__(self, players, log = True):
        self.players = players
        self.deck = getDeck()
        self.player_cards = getShuffled(self.deck)
        self.game_deck = self.player_cards[0] + self.player_cards[1]
        
        self.cheats = [0, 0]
        self.moves = [0, 0]
        self.checks = [0, 0]
        self.draw_decisions = [0, 0]
        
        for i, cards in zip([0, 1], self.player_cards):
            self.players[i].startGame(cards.copy())
            if log:
                print("Player (" + str(i + 1) + "): " + self.players[i].name + " received:")
                print(self.players[i].cards)
        
        ### Which card is on top
        self.true_card = None
        ### Which card was declared by active player
        self.declared_card = None
        
        ### Init pile: [-1] = top card
        self.pile = []
        
        ### Which player moves
        self.player_move = np.random.randint(2)
        
    def takeTurn(self, log = True):
        
        self.player_move = 1 - self.player_move
        
        if log: 
            print("")
            print("")
            print("==== CURRENT STATE ================================")
            print("==== " + self.players[self.player_move].name + " MOVES ====")
            print("Player (0): " + self.players[0].name + " hand:")
            print(self.players[0].cards)
            print("Player (1): " + self.players[1].name + " hand:")
            print(self.players[1].cards)
            print("Pile: ")
            print(self.pile)
            print("Declared top card:")
            print(self.declared_card)
            print("")
            
        activePlayer = self.players[self.player_move]
        opponent = self.players[1 - self.player_move]
        self.moves[self.player_move] += 1
        
        self.previous_declaration = self.declared_card
        decision = activePlayer.putCard(self.declared_card)
        
        if decision == "draw":
            
            if log: print("[+] " + activePlayer.name + " decides to draw cards")
            
            self.draw_decisions[self.player_move] += 1
            
            toTake = self.pile[max([-3, -len(self.pile)]):]
            for c in toTake: self.pile.remove(c)
            activePlayer.takeCards(toTake)
            for c in toTake: self.player_cards[self.player_move].append(c)
            
            self.declared_card = None
            self.true_card = None
            
            activePlayer.getCheckFeedback(False, False, False, None, None, log)
            opponent.getCheckFeedback(False, False, False, None, None, log)

        else:
            self.true_card, self.declared_card = decision
            if self.true_card != self.declared_card: self.cheats[self.player_move] += 1
            
            if log: print("[+] " + activePlayer.name + " puts " + str(self.true_card) +
                          " and declares " + str(self.declared_card))
        
            if not self.debugMove(): return False, self.player_move
        
            activePlayer.cards.remove(self.true_card)
            self.player_cards[self.player_move].remove(self.true_card) 
            self.pile.append(self.true_card)
        
            if opponent.checkCard(self.declared_card):
                
                self.checks[1 - self.player_move] += 1
                
                if log: print("[!] " + opponent.name + ": " + "I want to check")
                toTake = self.pile[max([-3, -len(self.pile)]):]
                for c in toTake: self.pile.remove(c)

                if not self.true_card == self.declared_card:
                    if log: print("\tYou are right!")
                    activePlayer.takeCards(toTake)
                
                    activePlayer.getCheckFeedback(True, False, True, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, False, tuple(toTake[-1]), len(toTake), log)
                
                    for c in toTake: self.player_cards[self.player_move].append(c)
                else:
                    if log: print("\tYou are wrong!")
                    opponent.takeCards(toTake)  
                
                    activePlayer.getCheckFeedback(True, False, False, None, len(toTake), log)
                    opponent.getCheckFeedback(True, True, True, tuple(toTake[-1]), len(toTake), log)
               
                    for c in toTake: self.player_cards[1 - self.player_move].append(c)
            
                if log:
                    print("Cards taken: ")
                    print(toTake)

                self.declared_card = None
                self.true_card = None
            else:
                activePlayer.getCheckFeedback(False, False, False, None, None, log)
                opponent.getCheckFeedback(False, False, False, None, None, log)

            
        if not self.debugGeneral(): return False, self.player_move
        return True, self.player_move
            
    def isFinished(self, log = True):
        if len(self.players[self.player_move].cards) == 0:
            if log: print(self.players[self.player_move].name + " wins!")
            return True
        return False

    def debugMove(self):
        if (self.previous_declaration is not None) and (self.true_card[0] < self.previous_declaration[0]) and \
                    len(self.players[self.player_move].cards) == 1:
            print("[ERROR] Last played card should be valid (it is revealed, you cannot cheat)!")
            return False
        if np.array(self.true_card).size != 2: 
            print("[ERROR] You put too many cards!")
            return False
        if self.true_card not in self.player_cards[self.player_move]:
            print("[ERROR] You do not have this card!")
            return False
        if self.true_card not in self.deck:
            print("[ERROR] There is no such card!")
            return False
        if (self.previous_declaration is not None) and len(self.pile) == 0:
            print("[ERROR] Inconsistency")
            return False
        if (self.previous_declaration is not None) and (self.declared_card[0] < self.previous_declaration[0]):
            print(len(self.pile))
            print(self.previous_declaration)
            print(self.declared_card)
            print(self.pile[-1])
            print("[ERROR] Improper move!")
            return False
        return True
    
    def debugGeneral(self):
        A = set(self.players[0].cards)
        B = set(self.players[1].cards)
        C = set(self.player_cards[0])
        D = set(self.player_cards[1])
        P = set(self.pile)
        E = set(self.game_deck)
        
        if not A == C: 
            print("Error 001")
            return False
        if not B == D:
            print("Error 002")
            return False
        if not A | B | P == E:
            print("Error 003")
            print(A)
            print(B)
            print(P)
            print(E)
            return False
        return True

Analyze few moves...

In [20]:
player1 = RandomPlayer("Player A")
player2 = RandomPlayer("Player B")
game = Game([player1, player2])

for i in range(6):
    game.takeTurn()

Player (1): Player A received:
[(9, 0), (11, 0), (11, 3), (10, 1), (14, 0), (14, 3), (13, 2), (9, 1)]
Player (2): Player B received:
[(11, 1), (12, 1), (9, 3), (14, 1), (13, 0), (12, 3), (13, 3), (10, 0)]


==== CURRENT STATE ================================
==== Player B MOVES ====
Player (0): Player A hand:
[(9, 0), (11, 0), (11, 3), (10, 1), (14, 0), (14, 3), (13, 2), (9, 1)]
Player (1): Player B hand:
[(11, 1), (12, 1), (9, 3), (14, 1), (13, 0), (12, 3), (13, 3), (10, 0)]
Pile: 
[]
Declared top card:
None

[+] Player B puts (9, 3) and declares (11, 1)
[!] Player A: I want to check
	You are right!
Feedback = Player B : checked this turn = True; I checked = False; I drew cards = True; revealed card = None; number of taken cards = 1
Feedback = Player A : checked this turn = True; I checked = True; I drew cards = False; revealed card = (9, 3); number of taken cards = 1
Cards taken: 
[(9, 3)]


==== CURRENT STATE ================================
==== Player A MOVES ====
Player (0): Play

In [21]:
### Some debug players
class DrawPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        return "draw"
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([False, False])
    
class SimplePlayer(Player):
    
    ### player's simple strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration
    
    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[0.3, 0.7])

In [22]:
### Perform a full game 100 times
stats_wins = [0, 0]
stats_moves = [0, 0]
stats_cheats = [0, 0]
stats_errors = [0, 0]
stats_cards = [0, 0]
stats_checks = [0, 0]
stats_draw_decisions = [0, 0]
stats_pile_size = 0

repeats = 100
errors = 0

for t in range(100):
    player1 = SimplePlayer("Player A")
    player2 = RandomPlayer("Player B")
    game = Game([player1, player2], log = False)
    
    error = False
    while True:
        valid, player = game.takeTurn(log = False)
        if not valid:
            error = True
            stats_errors[player] += 1
            errors += 1
            break
        if game.isFinished(log = False):
            stats_wins[player] += 1
            break
            
    stats_pile_size += len(game.pile)
    if not error:
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

stats_pile_size /= (repeats - errors)          
for j in range(2):
    stats_moves[j] /= (repeats - errors)
    stats_cheats[j] /= (repeats - errors)
    stats_checks[j] /= (repeats - errors)
    stats_draw_decisions[j] /= (repeats - errors)
    stats_cards[j] /= (repeats - errors)

    
print("Wins:")
print(stats_wins)
print("Moves:")
print(stats_moves)
print("Cards:")
print(stats_cards)
print("Pile size:")
print(stats_pile_size)
print("Checks:")
print(stats_checks)
print("Draw decisions:")
print(stats_draw_decisions)
print("Cheats:")
print(stats_cheats)
print("Errors:")
print(stats_errors)
print("Total errors:")
print(errors)

Wins:
[89, 11]
Moves:
[15.7, 15.37]
Cards:
[1.45, 12.71]
Pile size:
1.84
Checks:
[6.02, 7.52]
Draw decisions:
[0.0, 0.06]
Cheats:
[4.45, 9.08]
Errors:
[0, 0]
Total errors:
0


In [23]:
### Some debug players
class DrawPlayer(Player):

    ### player's random strategy
    def putCard(self, declared_card):
        return "draw"

    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([False, False])

class SimplePlayer(Player):

    ### player's simple strategy
    def putCard(self, declared_card):

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration

    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[0.3, 0.7])


class MySimplePlayer(Player):

    ### player's simple strategy
    def putCard(self, declared_card):

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration

    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return False

In [24]:
### Implement player's strategy. You can compare it with random player
### (or some strategy implemented by one of you colleagues)
### Time limit per decision 0.01s !!!

class YourFirstPlayer(Player):

    checkProb = 0.2
    shouldCheat = 0.001
    cheat = False
    randomlyPlayTopCard = 0.02

    ### player's random strategy
    def putCard(self, declared_card):
        # never return "draw" :))
        self.cards.sort(key=lambda x: x[0])
        self.cheat = False

        # check if maybe i want to randomly cheat
        # check if i have to cheat if no legal moves other than draw available

        ### NEVER DRAW - ale gra mówi, że ostatnia karta jest publiczna? huh?
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        # to nie pierwszy ruch, ale nie mam karty, ktora moge zagrac
        # i nie jest to ostatnia karta
        if len(self.cards) != 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            if np.random.choice([True, False], p=[self.shouldCheat, 1 - self.shouldCheat]):
                self.cheat = True

        # nie mam czego zagrac
        if len(self.cards) != 1 and declared_card is not None and self.cards[-1][0] < declared_card[0]:
            self.cheat = True

        # then play a card and declare a card
        if self.cheat:
            card = self.cards[0]
            if declared_card[0] < self.cards[-1][0]:
                declaration = (self.cards[-1][0], self.cards[-1][1]) # zmienic na randomowa z kart z reki, ktora > declared card
            else:
                # declaration = (self.cards[0][0], self.cards[0][1]) #TUUU był error
                declaration = (min(declared_card[0] + 1, 14), np.random.choice([0, 1, 2, 3])) # losowe kłamstwo
                # return "draw" # wow, ale najgorzej, ale to chyba sytuacja gdzie przeciwnik i tak widzi cały stosik, więc nie wiem czy jest sens kłamać. (tho kłamałbym for the record jak ktoś nie sprawdza)

        else:
            if np.random.choice([True, False], p=[self.randomlyPlayTopCard, 1 - self.randomlyPlayTopCard]):
                # randomly play the top card
                card = self.cards[-1]
                declaration = (self.cards[-1][0], self.cards[-1][1])
            else:
                # play the lowest possible card
                if declared_card is not None:
                    min_val = declared_card[0]
                    for lookForaCard in self.cards:
                        if lookForaCard[0] < min_val:
                            continue
                        else:
                            card = lookForaCard
                            declaration = (lookForaCard[0], lookForaCard[1])
                            break
                else:
                    # first turn - play the lowest card
                    card = self.cards[0]
                    declaration = (self.cards[0][0], self.cards[0][1])

        return card, declaration

    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[self.checkProb, 1 - self.checkProb])

In [25]:
class SimpleNoCheck(Player):

    ### player's simple strategy
    def putCard(self, declared_card):

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration

    def checkCard(self, opponent_declaration):
        return False


class SimpleAlwaysCheck(Player):

    ### player's simple strategy
    def putCard(self, declared_card):

        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        card = min(self.cards, key=lambda x: x[0])
        declaration = (card[0], card[1])
        if declared_card is not None:
            min_val = declared_card[0]
            if card[0] < min_val: declaration = (min(min_val + 1, 14), declaration[1])
        return card, declaration

    def checkCard(self, opponent_declaration):
        return True


class HonestPlayer(Player):

    def putCard(self, declared_card):

        # check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        self.cards.sort(key=lambda x: x[0])

        if declared_card is None:
            # play the lowest possible card
            card = self.cards[0]
            return card, (card[0], card[1])

        # honest player!
        if self.cards[-1][0] < declared_card[0]:
            return "draw"

        for card in self.cards:
            if card[0] >= declared_card[0]:
                return card, (card[0], card[1])

    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards: return True
        return np.random.choice([True, False], p=[0.3, 0.7])

In [27]:
class MySecondPlayer(Player):

    def __init__(self, name):

        super().__init__(name)
        # startowe wartości
        self.shouldCheat = 0.2
        self.checkProb = 0.2
        self.cheat = False
        self.randomlyPlayTopCard = 0.02
        self.learningRate = 0.3
        self.roundsCounter = 0
        # cardsOnPile = []

    # player's random strategy
    def putCard(self, declared_card):
        # never return "draw" :))
        self.cards.sort(key=lambda x: x[0])
        self.cheat = False

        # check if maybe i want to randomly cheat
        # check if i have to cheat if no legal moves other than draw available

        # NEVER DRAW - ale gra mówi, że ostatnia karta jest publiczna? huh?
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"

        # to nie pierwszy ruch, ale nie mam karty, ktora moge zagrac
        # i nie jest to ostatnia karta
        if len(self.cards) != 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            if np.random.choice([True, False], p=[self.shouldCheat, 1 - self.shouldCheat]):
                self.cheat = True

        # nie mam czego zagrac
        if len(self.cards) != 1 and declared_card is not None and self.cards[-1][0] < declared_card[0]:
            self.cheat = True

        # then play a card and declare a card
        if self.cheat:
            card = self.cards[0]
            if declared_card[0] < self.cards[-1][0]:
                declaration = (
                    self.cards[-1][0], self.cards[-1][1])  # zmienic na randomowa z kart z reki, ktora > declared card
            else:
                declaration = (min(declared_card[0] + 1, 14), np.random.choice([0, 1, 2, 3]))  # losowe kłamstwo

        # Play legit move
        else:
            if np.random.choice([True, False], p=[self.randomlyPlayTopCard, 1 - self.randomlyPlayTopCard]):
                # randomly play the top card
                card = self.cards[-1]
                declaration = (self.cards[-1][0], self.cards[-1][1])
            else:
                # play the lowest possible card
                if declared_card is not None:
                    min_val = declared_card[0]
                    for lookForaCard in self.cards:
                        if lookForaCard[0] < min_val:
                            continue
                        else:
                            card = lookForaCard
                            declaration = (lookForaCard[0], lookForaCard[1])
                            break
                else:
                    # first turn - play the lowest card
                    card = self.cards[0]
                    declaration = (self.cards[0][0], self.cards[0][1])

        return card, declaration

    # randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        if opponent_declaration in self.cards:
            return True
        return np.random.choice([True, False], p=[self.checkProb, 1 - self.checkProb])

    # Notification sent at the end of a round
    # One may implement this method, capture data, and use it to get extra info
    # -- checked = TRUE -> someone checked. If FALSE, the remaining inputs do not play any role
    # -- iChecked = TRUE -> I decided to check my opponent (so it was my turn);
    #               FALSE -> my opponent checked and it was his turn
    # -- iDrewCards = TRUE -> I drew cards (so I checked but was wrong or my opponent checked and was right);
    #                 FALSE -> otherwise
    # -- revealedCard - some card (X, Y). Only if I checked.
    # -- noTakenCards - number of taken cards
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log=True):
        self.roundsCounter += 1
        self.learningRate = 1 / self.roundsCounter
        # noTakenCards - użyć do pamiętania co jest na stosiku ;)
        if checked:
            if iChecked:
                # I Checked
                if iDrewCards:
                    # I checked and I was wrong, so I should check less often
                    self.checkProb = max(0.01, self.checkProb - self.learningRate)
                else:
                    # I checked and I was right, so I should check more often (?)
                    self.checkProb = min(0.99, self.checkProb + self.learningRate)
            else:
                # My opponent checked
                # so I should cheat less often
                self.shouldCheat = max(0.01, self.shouldCheat - self.learningRate)

        else:
            # No one checked
            # so maybe I should cheat more often?
            self.shouldCheat = min(0.99, self.shouldCheat + self.learningRate)  # do dopasowania wartości

        # print(f"Round: {self.roundsCounter}, checkProb: {self.checkProb}, shouldCheat: {self.shouldCheat}")

# W ogólności, to daje możliwości pamiętania ile wie mój przeciwnik i jakie karty widział etc.
# jeszcze tego nie zrobię, ale może się przydać,


In [31]:
### Perform a full game 100 times
stats_wins = [0, 0]
stats_moves = [0, 0]
stats_cheats = [0, 0]
stats_errors = [0, 0]
stats_cards = [0, 0]
stats_checks = [0, 0]
stats_draw_decisions = [0, 0]
stats_pile_size = 0

repeats = 1000
errors = 0

for t in range(repeats):
    player1 = MySecondPlayer("Player A")
    player2 = HonestPlayer("Player B")
    game = Game([player1, player2], log = False)

    error = False
    while True:
        valid, player = game.takeTurn(log = False)
        if not valid:
            error = True
            stats_errors[player] += 1
            errors += 1
            break
        if game.isFinished(log = False):
            stats_wins[player] += 1
            break

    stats_pile_size += len(game.pile)
    if not error:
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

stats_pile_size /= (repeats - errors)
for j in range(2):
    stats_moves[j] /= (repeats - errors)
    stats_cheats[j] /= (repeats - errors)
    stats_checks[j] /= (repeats - errors)
    stats_draw_decisions[j] /= (repeats - errors)
    stats_cards[j] /= (repeats - errors)


print("Wins:")
print(stats_wins)
print("Moves:")
print(stats_moves)
print("Cards:")
print(stats_cards)
print("Pile size:")
print(stats_pile_size)
print("Checks:")
print(stats_checks)
print("Draw decisions:")
print(stats_draw_decisions)
print("Cheats:")
print(stats_cheats)
print("Errors:")
print(stats_errors)
print("Total errors:")
print(errors)

Wins:
[859, 141]
Moves:
[11.979, 11.604]
Cards:
[1.326, 8.449]
Pile size:
6.225
Checks:
[1.023, 3.741]
Draw decisions:
[0.024, 1.233]
Cheats:
[2.64, 0.0]
Errors:
[0, 0]
Total errors:
0
